###Forward Propagation without Tensorflow

This notebook demonstrates forward propogation with the use of pretrained weights saved in a NumPy format as opposed to a Keras or Tensorflow format.

Once the forward propogation has been completed, a comparison between the predicted output and actual output will be conducted and the resulting accuracy will be returned.

In [ ]:
import numpy as np


class Inference():
    def __init__(self, input_data, actual_output):
        self.input_data = input_data
        self.actual_output = actual_output 
        self.run_inference = self.run(input_data, actual_output)


    def accuracy(self, pred_output, actual_output):
        correct = 0
        total = 0

        # Since pred_output is currently comprised of tensors with float values, 
        # the values must be converted to one hop format prior to comparison.
        one_hot_output = np.argmax(pred_output, axis=1)

        for idx, value in enumerate(actual_output):
            if value == one_hot_output[idx]:
                correct += 1
            total += 1

        return (f"Accuracy: {(correct / total) * 100}%")


    def relu_activation(self, pre_activation):
        '''
        ReLu activation will ensure that all values less than 0 from the dot 
        product calculation will be replaced with 0.
        '''
        return pre_activation * (pre_activation > 0)


    def calculate(self, curr_data, weights, bias):
        '''
        Here we are calculating the dot product of the input data and weights
        as well as adding a bias to the product.

        First we are pairing each value in the input tensor with a value in the weights.
        Next we are multiplying and summing the values in each data pair and adding a bias to the sum.
        '''
        weight_pairs = list(zip(*weights))
        out_arr = []

        for data in curr_data:
            sums = []
            for weight in weight_pairs:
                data_pairs = list(zip(data, weight))
                out_sum = sum([(x*y) for x,y in data_pairs])
                sums.append(out_sum)
            out_arr.append(sums)
        return out_arr + bias


    def run(self, input_data, actual_output):
        # Load all weights and biases
        d512w = np.load("d512w_new.npy")
        d512b = np.load("d512b_new.npy")
        d256w = np.load("d256w_new.npy")
        d256b = np.load("d256b_new.npy")
        d10w = np.load("d10w_new.npy")
        d10b = np.load("d10b_new.npy")

        # Propagate data through all layers
        dense_512 = self.calculate(input_data, d512w, d512b)
        dense_512_post = self.relu_activation(dense_512)
        dense_256 = self.calculate(dense_512_post, d256w, d256b)
        dense_256_post = self.relu_activation(dense_256)
        dense_10 = self.calculate(dense_256_post, d10w, d10b)

        # Calculate accuracy
        find_accuracy = self.accuracy(dense_10, actual_output)
        return find_accuracy



In [ ]:
# The initial Dense layer (Dense(512)) expects a flattened input, therefore the
# input data must be reshaped accordingly.
x_test_mod = np.load("x_test_mod.npy").reshape(-1, 28*28).astype("float32")
y_test_mod = np.load("y_test_mod.npy")

forward = Inference(x_test_mod, y_test_mod)
print(forward.run_inference)

Accuracy: 98.27%
